In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import sys, os
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool")
import denoise
from model import *
import torch.nn.functional as F

In [2]:
data = pd.read_csv("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/rssi/rssi_for_4AP.csv")

In [39]:
base_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/csi_dataset/RSSI/timestamp_allignment_Balanced_2024_12_14_rtt_logs.csv"

In [40]:
df = pd.read_csv(base_path)
df 

,timeStemp,Label,AP1_Distance (mm),AP2_Distance (mm),AP3_Distance (mm),AP4_Distance (mm),AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi,AP1_StdDev (mm),AP2_StdDev (mm),AP3_StdDev (mm),AP4_StdDev (mm)
0,493453295,1-11,718.0,8178.0,7432.0,4852.0,-60.0,-68.0,-61.0,-59.0,264.0,1502.0,325.0,934.0
1,493449532,1-11,639.0,8139.0,6930.0,4911.0,-62.0,-70.0,-59.0,-59.0,418.0,1466.0,799.0,485.0
2,493453815,1-11,678.0,8139.0,7471.0,4647.0,-61.0,-69.0,-62.0,-55.0,311.0,91.0,385.0,600.0
3,493452063,1-11,718.0,8022.0,7627.0,6219.0,-62.0,-69.0,-63.0,-55.0,324.0,1523.0,268.0,2649.0
4,493451194,1-11,561.0,8335.0,7471.0,4471.0,-61.0,-70.0,-62.0,-55.0,343.0,1195.0,312.0,1079.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19644,493967396,7-1,7280.0,4643.0,9678.0,1307.0,-64.0,-66.0,-61.0,-58.0,196.0,273.0,1174.0,230.0
19645,493961752,7-1,6309.0,2270.0,8503.0,844.0,-67.0,-75.0,-62.0,-59.0,125.0,174.0,1670.0,88.0
19646,493967789,7-1,6811.0,2475.0,9883.0,1375.0,-64.0,-66.0,-61.0,-57.0,377.0,918.0,822.0,188.0
19647,493968275,7-1,7837.0,2671.0,9511.0,897.0,-66.0,-65.0,-59.0,-58.0,594.0,1081.0,849.0,438.0


In [3]:
# Load data RSSI

label_to_coordinates_swapped = {
    "1-1":  (0, 0),   "1-2":  (0, 0.6),   "1-3":  (0, 1.2),   "1-4":  (0, 1.8),
    "1-5":  (0, 2.4), "1-6":  (0, 3.0),   "1-7":  (0, 3.6),   "1-8":  (0, 4.2),
    "1-9":  (0, 4.8), "1-10": (0, 5.4),   "1-11": (0, 6.0),

    "2-1":  (0.6, 0),  "2-11": (0.6, 6.0),

    "3-1":  (1.2, 0),  "3-11": (1.2, 6.0),

    "4-1":  (1.8, 0),  "4-11": (1.8, 6.0),

    "5-1":  (2.4, 0),  "5-11": (2.4, 6.0),

    "6-1":  (3.0, 0),  "6-2":  (3.0, 0.6),  "6-3":  (3.0, 1.2),  "6-4":  (3.0, 1.8),
    "6-5":  (3.0, 2.4), "6-6":  (3.0, 3.0),  "6-7":  (3.0, 3.6),  "6-8":  (3.0, 4.2),
    "6-9":  (3.0, 4.8), "6-10": (3.0, 5.4),  "6-11": (3.0, 6.0),

    "7-1":  (3.6, 0),  "7-11": (3.6, 6.0),

    "8-1":  (4.2, 0),  "8-11": (4.2, 6.0),

    "9-1":  (4.8, 0),  "9-11": (4.8, 6.0),

    "10-1": (5.4, 0),  "10-11": (5.4, 6.0),

    "11-1": (6.0, 0),  "11-2": (6.0, 0.6), "11-3": (6.0, 1.2), "11-4": (6.0, 1.8),
    "11-5": (6.0, 2.4),"11-6": (6.0, 3.0), "11-7": (6.0, 3.6), "11-8": (6.0, 4.2),
    "11-9": (6.0, 4.8),"11-10":(6.0, 5.4), "11-11":(6.0, 6.0)
}


coordinates = {
        1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
        36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),
        2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
        6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),
        12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
        16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
        20: (5.4, 6.0), 21: (6.0, 6.0),
        22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
        26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),
        41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
        44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
        47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
    }
coordinate_to_label2 = {value: key for key, value in coordinates.items()}

# label_to_coordinates:  "1-1" -> (0, 0)
# coordinates:          1      -> (0, 0)

# 1) 先建立一個「座標 -> 數字」的反轉字典
coord_to_num = {value: key for key, value in coordinates.items()}
# coord_to_num[(0,0)] = 1,  coord_to_num[(0.6,0)] = 40, ...

# 2) 建立 "1-1" -> 整數標籤 的 map
label_str_to_num = {}
for str_label, coord in label_to_coordinates_swapped.items():
    if coord in coord_to_num:  # 確保在座標字典中找得到
        label_str_to_num[str_label] = coord_to_num[coord]

In [43]:
df["Label"] = (
    df["Label"]
    .map(label_to_coordinates_swapped)   # 由 "1-1" → (0,0)
    .map(coordinate_to_label2)   # 由 (0,0) → 1 (或其他整數標籤)
)

In [45]:
data = df.drop(columns=['timeStemp', 'AP1_Distance (mm)', 'AP2_Distance (mm)', 'AP3_Distance (mm)', 'AP4_Distance (mm)', 'AP1_StdDev (mm)','AP2_StdDev (mm)', 'AP3_StdDev (mm)', 'AP4_StdDev (mm)'])

In [4]:
data

,Label,AP1_Rssi,AP2_Rssi,AP3_Rssi,AP4_Rssi
0,11,-60.0,-68.0,-61.0,-59.0
1,11,-62.0,-70.0,-59.0,-59.0
2,11,-61.0,-69.0,-62.0,-55.0
3,11,-62.0,-69.0,-63.0,-55.0
4,11,-61.0,-70.0,-62.0,-55.0
...,...,...,...,...,...
24495,35,-66.0,-65.0,-61.0,-59.0
24496,35,-67.0,-64.0,-62.0,-59.0
24497,35,-65.0,-65.0,-61.0,-60.0
24498,35,-65.0,-65.0,-61.0,-58.0


In [5]:
coordinates = {
        1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
        36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),
        2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
        6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),
        12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
        16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
        20: (5.4, 6.0), 21: (6.0, 6.0),
        22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
        26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),
        41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
        44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
        47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
    }

In [25]:
from torch.utils.data import Dataset
import torch

class RSSIDataset(Dataset):
    def __init__(self, rssi_data, labels, coordinates_dict, augment=False, rssi_mask_prob=0.05):
        self.rssi_data = torch.tensor(rssi_data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.float32)  # one-hot labels
        self.augment = augment
        self.rssi_mask_prob = rssi_mask_prob
        self.coordinates_dict = coordinates_dict

        # 將 class index（從 one-hot）反解為 ID（1~49）
        self.label_ids = torch.argmax(self.labels, dim=1).numpy() + 1

        # 對應到回歸座標
        self.reg_coords = torch.tensor(
            [coordinates_dict[int(lid)] for lid in self.label_ids],
            dtype=torch.float32
        )

    def __len__(self):
        return len(self.rssi_data)

    def __getitem__(self, idx):
        rssi = self.rssi_data[idx]
        label = self.labels[idx]              # one-hot
        reg_target = self.reg_coords[idx]     # 對應的 (x, y)

        if self.augment:
            mask = torch.rand(rssi.shape) < self.rssi_mask_prob
            rssi = rssi.clone()  # 避免修改原始資料
            rssi[mask] = 0.0

        return rssi, label, reg_target



In [26]:
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# 假設你已經有 df_merged_all
rssi_columns = ['AP1_Rssi', 'AP2_Rssi', 'AP3_Rssi', 'AP4_Rssi']

# 1. 填補 RSSI 缺值
data[rssi_columns] = data[rssi_columns].fillna(0)

# 2. Label 編碼（如果是整數，可以直接用）
labels = data['Label'].values
rssi_data = data[rssi_columns].values






In [27]:
rssi_data

array([[-60., -68., -61., -59.],
       [-62., -70., -59., -59.],
       [-61., -69., -62., -55.],
       ...,
       [-65., -65., -61., -60.],
       [-65., -65., -61., -58.],
       [-66., -64., -60., -59.]])

In [28]:
labels

array([11, 11, 11, ..., 35, 35, 35])

In [29]:
encoder = OneHotEncoder(sparse_output=False)
one_hot_labels = encoder.fit_transform(np.array(labels).reshape(-1, 1))

In [30]:
train_data, temp_data, train_labels, temp_labels = train_test_split(
    rssi_data, one_hot_labels, test_size=0.3, random_state=42)
val_data, test_data, val_labels, test_labels = train_test_split(
    temp_data, temp_labels, test_size=1/3, random_state=42)

In [31]:
train_dataset = RSSIDataset(train_data, train_labels,coordinates, augment=True)
val_dataset = RSSIDataset(val_data, val_labels, coordinates)
test_dataset = RSSIDataset(test_data, test_labels, coordinates)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [11]:
# for train_input, train_label in train_loader:
#     print(train_input.shape, train_label.shape)

In [13]:
# print(train_label[1])
# argmax_label = torch.argmax(train_label[1])
# print(argmax_label.item())

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [56]:
class RSSIClassifier(torch.nn.Module):
    def __init__(self, input_size, num_classes):
        super(RSSIClassifier, self).__init__()
        self.fc1 =torch.nn.Linear(input_size, 128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.dropout1 = torch.nn.Dropout(0.5)

        self.fc2 = torch.nn.Linear(128, 64)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.dropout2 = torch.nn.Dropout(0.5)

        self.fc3 = torch.nn.Linear(64, num_classes)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x


In [55]:
model = RSSIClassifier(4, num_classes=49).to(device)

In [34]:
from torchinfo import summary

model.eval()  # ✅ 切到推理模式避免 batchnorm 爆炸
dummy_rssi = torch.randn(32, 4).to(device)       # RSSI 輸入: (batch=1, 4-d)
output = model(dummy_rssi)
print("Output shape:", output.shape)  # 預期: (1, 49)

# 安裝 torchinfo (如果還沒安裝)
# pip install torchinfo

from torchinfo import summary
dummy_rssi = torch.randn(1, 4).to(device)
summary(model, input_data=dummy_rssi)


Output shape: torch.Size([32, 49])


Layer (type:depth-idx)                   Output Shape              Param #
RSSIClassifier                           [1, 49]                   --
├─Linear: 1-1                            [1, 128]                  640
├─BatchNorm1d: 1-2                       [1, 128]                  256
├─Dropout: 1-3                           [1, 128]                  --
├─Linear: 1-4                            [1, 64]                   8,256
├─BatchNorm1d: 1-5                       [1, 64]                   128
├─Dropout: 1-6                           [1, 64]                   --
├─Linear: 1-7                            [1, 49]                   3,185
Total params: 12,465
Trainable params: 12,465
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.05
Estimated Total Size (MB): 0.05

In [35]:
import torch.nn as nn
import torch.optim as optim

# 損失函數
criterion = nn.CrossEntropyLoss()

# 優化器
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 學習率調整器
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15, verbose=True)

/home/mcs/anaconda3/envs/kyle_ai/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [36]:
print(train_loader.dataset[0][0], train_loader.dataset[0][1])

tensor([-61., -66., -56., -52.]) tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [37]:
import torch
import matplotlib.pyplot as plt

# 儲存最佳模型相關設定
best_val_loss = float('inf')
best_model_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls_l2.pth"

# 訓練參數
epochs = 300

# Early Stopping 參數
patience = 20
counter = 0  

# 紀錄訓練過程中的 loss 和 accuracy
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(epochs):
    # ---- 訓練階段 ----
    model.train()
    train_loss = 0.0
    train_correct = 0
    total_train = 0

    for rssi_inputs, labels, _ in train_loader:
        rssi_inputs = rssi_inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(rssi_inputs)

        loss = criterion(outputs, torch.argmax(labels, dim=1))
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * rssi_inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total_train += labels.size(0)
        train_correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

    avg_train_loss = train_loss / len(train_loader.dataset)
    train_acc = 100 * train_correct / total_train

    # ---- 驗證階段 ----
    model.eval()
    val_loss = 0.0
    val_correct = 0
    total_val = 0

    with torch.no_grad():
        for rssi_inputs, labels, _ in val_loader:
            rssi_inputs = rssi_inputs.to(device)
            labels = labels.to(device)

            outputs = model(rssi_inputs)
            loss = criterion(outputs, torch.argmax(labels,dim=1))

            val_loss += loss.item() * rssi_inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            total_val += labels.size(0)
            val_correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

    avg_val_loss = val_loss / len(val_loader.dataset)
    val_acc = 100 * val_correct / total_val

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")

    # 儲存最佳模型
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model.state_dict(), best_model_path)
        print(f"✅ 儲存最佳模型 (Val Loss: {best_val_loss:.4f}) 至 {best_model_path}")
        counter = 0
    else:
        counter += 1

    if counter >= patience:
        print(f"⏹️ Early stop at epoch {epoch+1}")
        break

print("✅ 訓練完成！")


Epoch [1/300] | Train Loss: 3.4684 | Train Acc: 10.50% | Val Loss: 2.5244 | Val Acc: 40.80%
✅ 儲存最佳模型 (Val Loss: 2.5244) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls_l2.pth
Epoch [2/300] | Train Loss: 2.6628 | Train Acc: 23.04% | Val Loss: 1.7732 | Val Acc: 61.78%
✅ 儲存最佳模型 (Val Loss: 1.7732) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls_l2.pth
Epoch [3/300] | Train Loss: 2.3010 | Train Acc: 29.76% | Val Loss: 1.4795 | Val Acc: 68.22%
✅ 儲存最佳模型 (Val Loss: 1.4795) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls_l2.pth
Epoch [4/300] | Train Loss: 2.1407 | Train Acc: 33.86% | Val Loss: 1.3518 | Val Acc: 66.71%
✅ 儲存最佳模型 (Val Loss: 1.3518) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls_l2.pth
Epoch [5/300] | Train Loss: 2.0421 | Train Acc: 35.65% | Val Loss: 1.2394 | Val Acc: 71.63%
✅ 儲存最佳模型 (Val Loss: 1.2394) 

In [38]:
# 載入最佳模型
model.load_state_dict(torch.load("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls_l2.pth"))
model.eval()

# 測試模型
test_loss = 0.0
correct = 0
total = 0
all_labels = []
all_predictions = []

with torch.no_grad():
    for rssi_input, labels, _ in test_loader:
        rssi_inputs, labels = rssi_input.to(device), labels.to(device)
        outputs = model(rssi_inputs)
        loss = criterion(outputs, torch.argmax(labels, dim=1))
            
        test_loss += loss.item() * rssi_inputs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
        # 儲存真實標籤與預測標籤
        all_labels.extend(torch.argmax(labels, dim=1).cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

test_loss /= len(test_loader.dataset)
accuracy = 100 * correct / total

print(f"📊 測試損失: {test_loss:.4f}, 測試準確率: {accuracy:.2f}%")

📊 測試損失: 0.8309, 測試準確率: 78.61%


In [39]:
import numpy as np
adjusted_labels = np.array(all_labels) + 1
adjusted_predictions = np.array(all_predictions) + 1
def compute_mean_distance_error(y_true, y_pred, coordinates):
    """
    y_true, y_pred: 一維的 NumPy 陣列，分別存放真實和預測的 label（整數）
    coordinates: dict, label -> (x, y)
    """
    errors = []
    for true_label, pred_label in zip(y_true, y_pred):
        # 取出對應的座標
        if true_label not in coordinates or pred_label not in coordinates:
            # 若某個 label 不在座標字典內，就跳過（或視需求處理）
            print(f"Warning: Label {true_label} or {pred_label} not in coordinates.")
            continue
        true_coord = np.array(coordinates[true_label])
        pred_coord = np.array(coordinates[pred_label])
        # 計算歐氏距離
        error = np.linalg.norm(pred_coord - true_coord)
        errors.append(error)
    return np.mean(errors) if errors else None

COORDINATES = {
    # 下邊界 (1-10 和 40-31)
    1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
    36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),

    # 左邊界 (1-11)
    2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
    6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),

    # 上邊界 (11-21)
    12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
    16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
    20: (5.4, 6.0), 21: (6.0, 6.0),

    # 右邊界 (21-31)
    22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
    26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),

    # 中間點 (41-49)
    41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
    44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
    47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
}

mean_error = compute_mean_distance_error(adjusted_labels, adjusted_predictions, COORDINATES)
print("Mean distance error:", mean_error)


Mean distance error: 0.5880322996713262


# 5

In [57]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os
# 模擬資料和設定（請根據實際狀況載入這些）
# train_loader, val_loader, test_loader = ...
# COORDINATES = ...
# criterion = nn.CrossEntropyLoss()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_runs = 5
epochs = 300
patience = 20
input_size = 4
num_classes = 49

results = []

for run in range(1, num_runs + 1):
    print(f"\n=== Run {run}/{num_runs} ===")
    model = RSSIClassifier(input_size=input_size, num_classes=num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15)
    criterion = nn.CrossEntropyLoss()
    best_val_loss = float('inf')
    counter = 0

    for epoch in range(epochs):
        model.train()
        for rssi_inputs, labels, _ in train_loader:
            rssi_inputs, labels = rssi_inputs.to(device), labels.to(device)
            target_class = torch.argmax(labels, dim=1)
            optimizer.zero_grad()
            outputs = model(rssi_inputs)
            loss = criterion(outputs, target_class)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for rssi_inputs, labels, _ in val_loader:
                rssi_inputs, labels = rssi_inputs.to(device), labels.to(device)
                target_class = torch.argmax(labels, dim=1)
                outputs = model(rssi_inputs)
                loss = criterion(outputs, target_class)
                val_loss += loss.item() * rssi_inputs.size(0)

        val_loss /= len(val_loader.dataset)
        scheduler.step(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
        else:
            counter += 1
            if counter >= patience:
                break

    # 測試階段
    model.eval()
    all_true, all_pred = [], []
    with torch.no_grad():
        for rssi_inputs, labels, _ in test_loader:
            rssi_inputs, labels = rssi_inputs.to(device), labels.to(device)
            target_class = torch.argmax(labels, dim=1)
            outputs = model(rssi_inputs)
            predicted = torch.argmax(outputs, dim=1)
            all_true.extend(target_class.cpu().numpy())
            all_pred.extend(predicted.cpu().numpy())

    y_true = np.array(all_true) + 1
    y_pred = np.array(all_pred) + 1
    acc = 100 * np.mean(y_true == y_pred)
    mde = compute_mean_distance_error(y_true, y_pred, COORDINATES)

    print(f"✅ Run {run}: Accuracy = {acc:.2f}%, MDE = {mde:.4f}")
    results.append({'run': run, 'accuracy': acc, 'mde': mde})

# 儲存 CSV
os.makedirs("repeat/00", exist_ok=True)
df = pd.DataFrame(results)
df.to_csv("repeat/00/rssi_cls_results00.csv", index=False)



=== Run 1/5 ===
✅ Run 1: Accuracy = 76.94%, MDE = 0.6276

=== Run 2/5 ===
✅ Run 2: Accuracy = 72.86%, MDE = 0.7204

=== Run 3/5 ===
✅ Run 3: Accuracy = 78.33%, MDE = 0.5916

=== Run 4/5 ===
✅ Run 4: Accuracy = 77.63%, MDE = 0.5940

=== Run 5/5 ===
✅ Run 5: Accuracy = 78.49%, MDE = 0.5877


# 雙輸出

In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class RSSIClsRegNet(nn.Module):
    def __init__(self, input_size, num_classes):
        super(RSSIClsRegNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.bn1 = nn.BatchNorm1d(128)
        self.dropout1 = nn.Dropout(0.5)

        self.fc2 = nn.Linear(128, 64)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.5)

        # 分支 1：分類
        self.classifier = nn.Linear(64, num_classes)

        # 分支 2：回歸 (預測 X, Y)
        self.regressor = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)

        cls_out = self.classifier(x)  # 分類輸出
        reg_out = self.regressor(x)   # 回歸輸出 (x, y 座標)

        return cls_out, reg_out


In [47]:
model = RSSIClsRegNet(input_size=4, num_classes=49).to(device)

In [48]:
output = model(rssi_inputs)
print(type(output))


<class 'tuple'>


In [142]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

alpha = 0.5  # 回歸損失權重
epochs = 200
best_val_loss = float('inf')
best_model_path = "/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls_reg_05.pth"
patience = 20
counter = 0
save_enabled = True

optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15, verbose=True)

# 儲存訓練記錄
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

# Loss 定義
criterion_cls = nn.CrossEntropyLoss()
criterion_reg = nn.MSELoss()

# Label ID → 座標對應函數
def labels_to_coords(label_tensor, coord_dict):
    coords = [coord_dict[int(l.item()) + 1] for l in label_tensor.cpu()]
    return torch.tensor(coords, dtype=torch.float32, device=label_tensor.device)

# 訓練迴圈
for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    train_class_loss = 0.0
    train_reg_loss = 0.0
    train_correct = 0
    total_train = 0

    for rssi_inputs, labels,_ in train_loader:
        rssi_inputs = rssi_inputs.to(device)
        labels = labels.to(device)
        target_class = torch.argmax(labels, dim=1)

        optimizer.zero_grad()
        class_out, reg_out = model(rssi_inputs)

        loss_cls = criterion_cls(class_out, target_class)
        true_coords = labels_to_coords(target_class, COORDINATES)
        loss_reg = criterion_reg(reg_out, true_coords)
        loss = loss_cls + alpha * loss_reg

        loss.backward()
        optimizer.step()

        batch_size = rssi_inputs.size(0)
        train_loss += loss.item() * batch_size
        train_class_loss += loss_cls.item() * batch_size
        train_reg_loss += loss_reg.item() * batch_size

        _, predicted = torch.max(class_out, 1)
        train_correct += (predicted == target_class).sum().item()
        total_train += batch_size

    avg_train_loss = train_loss / len(train_loader.dataset)
    avg_train_class_loss = train_class_loss / len(train_loader.dataset)
    avg_train_reg_loss = train_reg_loss / len(train_loader.dataset)
    train_acc = 100 * train_correct / total_train

    # ----- 驗證 -----
    model.eval()
    val_loss = 0.0
    val_class_loss = 0.0
    val_reg_loss = 0.0
    val_correct = 0
    total_val = 0

    with torch.no_grad():
        for rssi_inputs, labels, _ in val_loader:
            rssi_inputs = rssi_inputs.to(device)
            labels = labels.to(device)
            target_class = torch.argmax(labels, dim=1)

            class_out, reg_out = model(rssi_inputs)
            loss_cls = criterion_cls(class_out, target_class)
            true_coords = labels_to_coords(target_class, COORDINATES)
            loss_reg = criterion_reg(reg_out, true_coords)
            loss = loss_cls + alpha * loss_reg

            batch_size = rssi_inputs.size(0)
            val_loss += loss.item() * batch_size
            val_class_loss += loss_cls.item() * batch_size
            val_reg_loss += loss_reg.item() * batch_size

            _, predicted = torch.max(class_out, 1)
            val_correct += (predicted == target_class).sum().item()
            total_val += batch_size

    avg_val_loss = val_loss / len(val_loader.dataset)
    avg_val_class_loss = val_class_loss / len(val_loader.dataset)
    avg_val_reg_loss = val_reg_loss / len(val_loader.dataset)
    val_acc = 100 * val_correct / total_val

    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    scheduler.step(avg_val_loss)

    print(f"Epoch [{epoch+1}/{epochs}] | "
          f"Train Loss: {avg_train_loss:.4f} (Cls: {avg_train_class_loss:.4f}, Reg: {avg_train_reg_loss:.4f}) | "
          f"Train Acc: {train_acc:.2f}% || "
          f"Val Loss: {avg_val_loss:.4f} (Cls: {avg_val_class_loss:.4f}, Reg: {avg_val_reg_loss:.4f}) | "
          f"Val Acc: {val_acc:.2f}%")

    # 儲存模型
    if save_enabled:
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), best_model_path)
            print(f"✅ 儲存最佳模型 (Val Loss: {best_val_loss:.4f}) 至 {best_model_path}")
            counter = 0
        else:
            counter += 1
            print(f"⏳ 沒改善: {counter}/{patience}")
            if counter >= patience:
                print(f"⚠️ 已達 patience={patience}，提前停止訓練")
                break

print("🎉 訓練完成！")


/home/mcs/anaconda3/envs/kyle_ai/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch [1/200] | Train Loss: 2.5891 (Cls: 1.5232, Reg: 2.1318) | Train Acc: 50.69% || Val Loss: 1.7618 (Cls: 0.9438, Reg: 1.6360) | Val Acc: 74.47%
✅ 儲存最佳模型 (Val Loss: 1.7618) 至 /media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls_reg_05.pth


KeyboardInterrupt: 

In [ ]:
# 載入最佳模型
model.load_state_dict(torch.load("/media/mcs/1441ae67-d7cd-43e6-b028-169f78661a2f/kyle/csi_tool/model_save_rssi_dnn/rssi_cls_reg_06.pth"))
model.eval()

# 測試模型
test_loss = 0.0
correct = 0
total = 0

all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels, _ in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        class_out, _ = model(inputs)  # 只拿分類頭
        loss = criterion(class_out, torch.argmax(labels, dim=1))
        test_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(class_out, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()
        all_labels.extend(torch.argmax(labels, dim=1).cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())



test_loss /= len(test_loader.dataset)
accuracy = 100 * correct / total

print(f"📊 測試損失: {test_loss:.4f}, 測試準確率: {accuracy:.2f}%")

📊 測試損失: 0.9236, 測試準確率: 77.47%


In [45]:
import numpy as np
adjusted_labels = np.array(all_labels) + 1
adjusted_predictions = np.array(all_predictions) + 1
def compute_mean_distance_error(y_true, y_pred, coordinates):
    """
    y_true, y_pred: 一維的 NumPy 陣列，分別存放真實和預測的 label（整數）
    coordinates: dict, label -> (x, y)
    """
    errors = []
    for true_label, pred_label in zip(y_true, y_pred):
        # 取出對應的座標
        if true_label not in coordinates or pred_label not in coordinates:
            # 若某個 label 不在座標字典內，就跳過（或視需求處理）
            print(f"Warning: Label {true_label} or {pred_label} not in coordinates.")
            continue
        true_coord = np.array(coordinates[true_label])
        pred_coord = np.array(coordinates[pred_label])
        # 計算歐氏距離
        error = np.linalg.norm(pred_coord - true_coord)
        errors.append(error)
    return np.mean(errors) if errors else None

COORDINATES = {
    # 下邊界 (1-10 和 40-31)
    1: (0, 0), 40: (0.6, 0), 39: (1.2, 0), 38: (1.8, 0), 37: (2.4, 0),
    36: (3.0, 0), 35: (3.6, 0), 34: (4.2, 0), 33: (4.8, 0), 32: (5.4, 0), 31: (6.0, 0),

    # 左邊界 (1-11)
    2: (0, 0.6), 3: (0, 1.2), 4: (0, 1.8), 5: (0, 2.4),
    6: (0, 3.0), 7: (0, 3.6), 8: (0, 4.2), 9: (0, 4.8), 10: (0, 5.4), 11: (0, 6.0),

    # 上邊界 (11-21)
    12: (0.6, 6.0), 13: (1.2, 6.0), 14: (1.8, 6.0), 15: (2.4, 6.0),
    16: (3.0, 6.0), 17: (3.6, 6.0), 18: (4.2, 6.0), 19: (4.8, 6.0),
    20: (5.4, 6.0), 21: (6.0, 6.0),

    # 右邊界 (21-31)
    22: (6.0, 5.4), 23: (6.0, 4.8), 24: (6.0, 4.2), 25: (6.0, 3.6),
    26: (6.0, 3.0), 27: (6.0, 2.4), 28: (6.0, 1.8), 29: (6.0, 1.2), 30: (6.0, 0.6),

    # 中間點 (41-49)
    41: (3.0, 0.6), 42: (3.0, 1.2), 43: (3.0, 1.8),
    44: (3.0, 2.4), 45: (3.0, 3.0), 46: (3.0, 3.6),
    47: (3.0, 4.2), 48: (3.0, 4.8), 49: (3.0, 5.4)
}

mean_error = compute_mean_distance_error(adjusted_labels, adjusted_predictions, COORDINATES)
print("Mean distance error:", mean_error)


Mean distance error: 0.5880322996713262


In [50]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

alphas = np.arange(0.1, 1.1, 0.1)
num_runs = 5
epochs = 300
patience = 20
# Loss 定義
criterion_cls = nn.CrossEntropyLoss()
criterion_reg = nn.MSELoss()

def labels_to_coords(label_tensor, coord_dict):
    coords = [coord_dict[int(l.item()) + 1] for l in label_tensor.cpu()]
    return torch.tensor(coords, dtype=torch.float32, device=label_tensor.device)

summary_results = []

for alpha in alphas:
    test_accs = []
    test_mdes = []

    print(f"\n🔁 Running alpha = {alpha:.1f} ...")

    for run in range(1, num_runs + 1):
        print(f"\n=== Run {run}/{num_runs} | Alpha = {alpha:.1f} ===")

        model = RSSIClsRegNet(input_size=4, num_classes=49).to(device)
        optimizer = optim.Adam(model.parameters(), lr=1e-3)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=15)
        best_val_loss = float('inf')
        counter = 0

        # Training
        for epoch in range(epochs):
            model.train()
            for rssi_inputs, labels, _ in train_loader:
                rssi_inputs, labels = rssi_inputs.to(device), labels.to(device)
                target_class = torch.argmax(labels, dim=1)
                optimizer.zero_grad()
                class_out, reg_out = model(rssi_inputs)
                loss_cls = criterion(class_out, target_class)
                loss_reg = criterion_reg(reg_out, labels_to_coords(target_class, COORDINATES))
                loss = loss_cls + alpha * loss_reg
                loss.backward()
                optimizer.step()

            # Validation
            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for rssi_inputs, labels, _ in val_loader:
                    rssi_inputs, labels = rssi_inputs.to(device), labels.to(device)
                    target_class = torch.argmax(labels, dim=1)
                    class_out, reg_out = model(rssi_inputs)
                    loss_cls = criterion(class_out, target_class)
                    loss_reg = criterion_reg(reg_out, labels_to_coords(target_class, COORDINATES))
                    val_loss += (loss_cls + alpha * loss_reg).item() * rssi_inputs.size(0)

            val_loss /= len(val_loader.dataset)
            scheduler.step(val_loss)

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                counter = 0
            else:
                counter += 1
                if counter >= patience:
                    break

        # Testing
        model.eval()
        all_true, all_pred = [], []
        with torch.no_grad():
            for rssi_inputs, labels, _ in test_loader:
                rssi_inputs, labels = rssi_inputs.to(device), labels.to(device)
                target_class = torch.argmax(labels, dim=1)
                class_out, _ = model(rssi_inputs)
                pred = torch.argmax(class_out, dim=1)
                all_pred.extend(pred.cpu().numpy())
                all_true.extend(target_class.cpu().numpy())

        y_true = np.array(all_true) + 1
        y_pred = np.array(all_pred) + 1
        acc = 100 * np.mean(y_true == y_pred)
        mde = compute_mean_distance_error(y_true, y_pred, COORDINATES)

        test_accs.append(acc)
        test_mdes.append(mde)

        print(f"✅ Run {run}: Acc = {acc:.2f}%, MDE = {mde:.4f}")

    # 儲存每一組 alpha 結果
    alpha_id = int(round(alpha * 10))  # e.g., 0.1 → 1
    folder_name = f"repeat/{alpha_id:02d}"
    os.makedirs(folder_name, exist_ok=True)

    df_runs = pd.DataFrame({
        'run': list(range(1, num_runs+1)),
        'accuracy': test_accs,
        'mde': test_mdes
    })
    df_runs.to_csv(f"{folder_name}/rssi_cls_reg_results{alpha_id:02d}.csv", index=False)
    print(f"📁 Saved run results to {folder_name}/rssi_cls_reg_results{alpha_id:02d}.csv")

    # 總結統計
    summary_results.append({
        'alpha': alpha,
        'avg_acc': np.mean(test_accs),
        'std_acc': np.std(test_accs),
        'avg_mde': np.mean(test_mdes),
        'std_mde': np.std(test_mdes)
    })



🔁 Running alpha = 0.1 ...

=== Run 1/5 | Alpha = 0.1 ===
✅ Run 1: Acc = 78.24%, MDE = 0.5940

=== Run 2/5 | Alpha = 0.1 ===
✅ Run 2: Acc = 74.78%, MDE = 0.6586

=== Run 3/5 | Alpha = 0.1 ===
✅ Run 3: Acc = 74.37%, MDE = 0.6611

=== Run 4/5 | Alpha = 0.1 ===
✅ Run 4: Acc = 74.57%, MDE = 0.6820

=== Run 5/5 | Alpha = 0.1 ===
✅ Run 5: Acc = 77.63%, MDE = 0.6082
📁 Saved run results to repeat/01/rssi_cls_reg_results01.csv

🔁 Running alpha = 0.2 ...

=== Run 1/5 | Alpha = 0.2 ===
✅ Run 1: Acc = 78.08%, MDE = 0.5824

=== Run 2/5 | Alpha = 0.2 ===
✅ Run 2: Acc = 77.59%, MDE = 0.5999

=== Run 3/5 | Alpha = 0.2 ===
✅ Run 3: Acc = 78.65%, MDE = 0.5942

=== Run 4/5 | Alpha = 0.2 ===
✅ Run 4: Acc = 76.78%, MDE = 0.6436

=== Run 5/5 | Alpha = 0.2 ===
✅ Run 5: Acc = 78.45%, MDE = 0.5819
📁 Saved run results to repeat/02/rssi_cls_reg_results02.csv

🔁 Running alpha = 0.3 ...

=== Run 1/5 | Alpha = 0.3 ===
✅ Run 1: Acc = 76.82%, MDE = 0.6039

=== Run 2/5 | Alpha = 0.3 ===
✅ Run 2: Acc = 77.55%, MDE = 0.

In [53]:
summary_results

[{'alpha': np.float64(0.1),
  'avg_acc': np.float64(75.91836734693877),
  'std_acc': np.float64(1.6659862556700866),
  'avg_mde': np.float64(0.6407665534337206),
  'std_mde': np.float64(0.033702356225171)},
 {'alpha': np.float64(0.2),
  'avg_acc': np.float64(77.91020408163266),
  'std_acc': np.float64(0.6723676826767033),
  'avg_mde': np.float64(0.6004259424645325),
  'std_mde': np.float64(0.02268343538980227)},
 {'alpha': np.float64(0.30000000000000004),
  'avg_acc': np.float64(76.9795918367347),
  'std_acc': np.float64(0.3414938883812578),
  'avg_mde': np.float64(0.6029104884298587),
  'std_mde': np.float64(0.0010298055062205073)},
 {'alpha': np.float64(0.4),
  'avg_acc': np.float64(77.28163265306122),
  'std_acc': np.float64(0.3763964696619922),
  'avg_mde': np.float64(0.6089817124915717),
  'std_mde': np.float64(0.021670374542471536)},
 {'alpha': np.float64(0.5),
  'avg_acc': np.float64(76.40816326530611),
  'std_acc': np.float64(0.6296836171649582),
  'avg_mde': np.float64(0.63509